<a href="https://colab.research.google.com/github/ArsT1982/Spark_DZ2/blob/main/Spark_SQL_DZ_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ДЗ-2 (Spark SQL)


---



Подготовка к работе с файлом "covid-data.csv" и создание Spark сессии.

In [2]:
pip install --quiet pyspark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Practice").getOrCreate()

In [5]:
from pyspark import SparkFiles

covid_data_file_url = "https://raw.githubusercontent.com/glincow/netology-spark-sql/main/data/covid-data.csv"
spark.sparkContext.addFile(covid_data_file_url)
file_path  = 'file://' + SparkFiles.get('covid-data.csv')
df = spark.read.option('inferSchema', 'true').option('header', 'true').csv(file_path)

Подготовка датасета к работе:
Анализ данных и значений в исходном файле.
Обнаружил:
1. В файле присутствуют null - заменяем эти значения на 0.
2. В файле присутствуют данные по Евросоюзу (и др. OWID_% варианты в поле 'iso_code').
Для аналитики важны данные по страны - их и оставил, отсальное исключил.


In [6]:
Find_OWID_from_df = df.filter(F.col('iso_code').like('OWID_%'))
Exclude_OWID_from_df = df.subtract(Find_OWID_from_df)
#Exclude_OWID_from_df.show()

df_null_to_zero=Exclude_OWID_from_df.na.fill(0)
#df_null_to_zero.show()


Задание-1.


---

Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших).


---
Важно! Посчитал, что переболевшие (alias 'recovered') - это те, кто входит в кол-во в графе общего количества заболевших ('total_cases'), но не относится к графе умерших ('total_deaths'). ("Переболел - значит выжил")

Посчитал как частное от деления на популяцию, умноженное на 100 (для процентов) и округлил до второго знака.
Данные по Евросоюзу были исколючены на предыдущих шагах.

In [ ]:
from pyspark.sql import functions as F

df_perc_recov_by_cntry = df_null_to_zero.select('iso_code', 'location',F.round(((F.col('total_cases')-F.col('total_deaths'))*100.0/F.col('population')),2).alias('recovered,%')).filter(df_null_to_zero.date == '2020-03-31').sort(F.col('recovered,%').desc())
df_perc_recov_by_cntry.show(15)

+--------+-------------+-----------+
|iso_code|     location|recovered,%|
+--------+-------------+-----------+
|     VAT|      Vatican|       0.74|
|     SMR|   San Marino|       0.62|
|     AND|      Andorra|       0.47|
|     LUX|   Luxembourg|       0.34|
|     ISL|      Iceland|       0.33|
|     CHE|  Switzerland|       0.19|
|     ESP|        Spain|       0.19|
|     LIE|Liechtenstein|       0.18|
|     ITA|        Italy|       0.15|
|     MCO|       Monaco|       0.13|
|     AUT|      Austria|       0.11|
|     BEL|      Belgium|        0.1|
|     DEU|      Germany|       0.08|
|     NOR|       Norway|       0.08|
|     FRA|       France|       0.07|
+--------+-------------+-----------+
only showing top 15 rows



Задание-2.


---

Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)


---
Посчитал в двух вариантах (т.к. Великий Могучий Русский Язык и можно задание интерпретировать по-разному).

Вариант-1:
Посчитать Топ 10 стран с самыми большими вспышками (F.max) новых случаев в течении последней (4-ой) недели марта 2021 (по убыванию).

Вариант-2:
Посчитать Топ 10 стран с самым большим суммарным количеством новых случаев (F.sum) случаев в течении последней (4-ой) недели марта 2021 (по убыванию).



Вариант-1.

---

Отфильровал весь файл по нужным нам датам 4ой недели марта 2021.
Нашел максимумы (F.max) аггрегатной функции в группировке по странам.
Сортировка по убыванию, отображение 10-ти стран.

In [ ]:
from pyspark.sql import functions as F

df_new_cases_week = df_null_to_zero.select('iso_code', 'location', 'date', 'new_cases').filter(df_null_to_zero.date.between ('2021-03-22','2021-03-28')).groupBy("location").agg(F.max("new_cases").alias("max new cases")).sort(F.col('max new cases').desc())
df_new_cases_week.show(10)

+-------------+-------------+
|     location|max new cases|
+-------------+-------------+
|       Brazil|     100158.0|
|United States|      86960.0|
|        India|      68020.0|
|       France|      65392.0|
|       Poland|      35145.0|
|       Turkey|      30021.0|
|        Italy|      24501.0|
|      Germany|      23757.0|
|         Peru|      19206.0|
|      Ukraine|      18226.0|
+-------------+-------------+
only showing top 10 rows



Вариант-2.

---

Отфильровал весь файл по нужным нам датам 4ой недели марта 2021.
Нашел суммарные значения (F.sum) аггрегатной функции в группировке по странам.
Сортировка по убыванию, отображение 10-ти стран.

In [21]:
from pyspark.sql import functions as F

df_new_cases_week = df_null_to_zero.select('iso_code', 'location', 'date', 'new_cases').filter(df_null_to_zero.date.between ('2021-03-22','2021-03-28')).groupBy("location").agg(F.sum("new_cases").alias("max sum new cases")).sort(F.col('max sum new cases').desc())
df_new_cases_week.show(10)

+-------------+-----------------+
|     location|max sum new cases|
+-------------+-----------------+
|       Brazil|         536455.0|
|United States|         442739.0|
|        India|         393563.0|
|       France|         263088.0|
|       Turkey|         195051.0|
|       Poland|         192441.0|
|        Italy|         155681.0|
|      Germany|         114651.0|
|      Ukraine|          98367.0|
|         Peru|          63556.0|
+-------------+-----------------+
only showing top 10 rows



Как видим - список стран в результатах отличается.
Первый список говорит о пиковых значениях (вспышках за данный период, который может коррелировать с событиями - нехватка вакцин, открытие аэропортов, снятие ограничений и т.д.).

Второй список стран показывает страны с затяжной тяжелой обстановкой c Covid-19. То есть факторы имеют влияние на более длительном периоде и более репрезентативные с точки зрения глобальных факторов (условно-постоянных) в странах (плотность населения (высокая),  охват вакцинацией (недостаточный),  качество медицины (низкое) и др.)

Задание-3.

Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

Использовал оконную функцию lag для нахождения значения за предыдущий день (alias 'yesterday_cases').
Для корректного расчета первого дня в выборке в исходном селекте использовал более шировкий диапазон дат - с 21 марта для исключения нуля в результате ф-ции lag.
В итоговом результате оставлен период за 4ю неделю с 22 по 28 марта 2021г.
Разница с предыдущим днем - alias 'delta'

In [19]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

w = Window.orderBy("date")
select_for_df = df_null_to_zero.select('iso_code', 'location', 'date', 'new_cases').filter(df_null_to_zero.date.between ('2021-03-21','2021-03-28')).filter(F.col('location').like('Russia')).sort(F.col('date').asc())

df_itog = select_for_df.withColumn("yesterday_cases", lag('new_cases', 1, 0).over(w))
df_result = df_itog.select('date','new_cases','yesterday_cases',(F.col('new_cases')-F.col('yesterday_cases')).alias('delta')).filter(df_null_to_zero.date.between ('2021-03-22','2021-03-28')).show()


+----------+---------+---------------+------+
|      date|new_cases|yesterday_cases| delta|
+----------+---------+---------------+------+
|2021-03-22|   9195.0|         9215.0| -20.0|
|2021-03-23|   8369.0|         9195.0|-826.0|
|2021-03-24|   8769.0|         8369.0| 400.0|
|2021-03-25|   9128.0|         8769.0| 359.0|
|2021-03-26|   9073.0|         9128.0| -55.0|
|2021-03-27|   8783.0|         9073.0|-290.0|
|2021-03-28|   8979.0|         8783.0| 196.0|
+----------+---------+---------------+------+

